- summary 
- web : server-client : url 
- requests : 동적페이지 : URL 변화 없이 페이지의 데이터 수정 : json(str) > response.json() > DataFrame 
- requests : 정적페이지 : URL 변화 있이 페이지의 데이터 수정 : html(str) > BeautifulSoup > css-selector > DataFrame 
- selenium : 웹브라우져를 python 코드로 컨트롤해서 데이터 수집 
- requests(동적페이지,API) > requests(정적페이지) > selenium 
- 웹크롤링 절차 
- 1. 웹서비스분석(개발자도구), API문서 : URL 
- 2. request(URL) > response(data) : data(json(str), html(str)) 
- 3. data(json(str), html(str)) > response.json(), BeautifulSoup(css-selector) > DataFrame 
- request 할때 401,403,500 등등의 에러가 발생하는 경우 > headers 수정해서 데이터 요청(user-agent, refere) 
- API 이용 : request token 수집후 크롤링

### Zigbang 원룸 매물 데이터 수집

##### 동 이름 입력하면 > 해당 동의 매물 데이터가 나오겠금

In [61]:
import pandas as pd
import requests

#### Process
    - 동이름으로 위도 경도 구하기(직빵엔 매물 데이터가 없음...)
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

- address url은 https://meyerweb.com/eric/tools/dencoder/ 에서 decode 해가지고 가져왔음

In [62]:
# 1. URL
address = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸'
# 2. request > response : json(str)
response = requests.get(url)
# 3. json(str) > lat, lng(위도값, 경도값)

In [63]:
response.text[:500]

'{"success":true,"code":"200","items":[{"id":4002,"type":"address","name":"망원동","hint":"","description":"서울시 마포구 망원동","lat":37.556785583496094,"lng":126.9013442993164,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"서울시","local2":"마포구","local3":"망원동","web_level":15,"web_lat":37.556785583496094,"web_lng":126.9013442993164,"app_level":15,"app_lat":37.556785583496094,"app_lng":126.9013442993164,"법정동코드":"1144012300"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5'

In [64]:
# list > dict > items만 가져오기
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

#### 2. 위도 경도로 geohash 알아내기
- geohash2 설치해야 함 
- 설치 방법 : pip install geohash2 (terminal > cli 환경에서 실행 comand line interface) 걍 터미널 켜서 하셈
- 그 후 임포트

In [65]:
import geohash2

In [66]:
# 작은 문자열로 나누면 더 많은 코드값, 문자열이 필요함
# precision : 클수록 영역이 작아짐 
geohash = geohash2.encode(lat, lng, precision=5) 
geohash

'wydjx'

#### 3. geohash로 매물 아이디 가져오기

In [67]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
response

<Response [200]>

In [48]:
# data = response.json()['items']
# data

[{'lat': 35.810173336459904, 'lng': 128.54918001281942, 'item_id': 35237240}]

In [68]:
data = response.json()['items']
ids = [item['item_id'] for item in data] # 아이템 아이디만 가져오기
len(ids), ids[:5]

(675, [35471881, 35346672, 35365643, 35070248, 35265682])

#### 4. 매물 아이디로 매물 정보 가져오기

In [69]:
ids

[35471881,
 35346672,
 35365643,
 35070248,
 35265682,
 35411116,
 35403433,
 35472522,
 35059106,
 35269309,
 35420328,
 35441970,
 35419933,
 35420661,
 35455652,
 35363643,
 35357900,
 35431532,
 35448617,
 35454937,
 35464605,
 35471885,
 35035055,
 35377237,
 35433115,
 35247257,
 35420705,
 35440070,
 35445413,
 35454950,
 35456089,
 35471927,
 35231837,
 35371199,
 35228887,
 35308996,
 35201972,
 35277659,
 35434908,
 35454005,
 35460297,
 35469009,
 35091891,
 35175569,
 35424938,
 35454146,
 35385157,
 35136006,
 35402634,
 35421103,
 35337288,
 35195337,
 35241262,
 35310213,
 35347421,
 35362340,
 34943073,
 35424163,
 35094756,
 35310153,
 35464253,
 34942970,
 35121907,
 34564597,
 34781572,
 34969984,
 35048671,
 35079128,
 35344276,
 35405058,
 35440554,
 35018418,
 35330199,
 35340579,
 35439142,
 35435610,
 35445702,
 35145746,
 35353300,
 35210886,
 35278630,
 35351667,
 35345163,
 35400024,
 35447414,
 35048409,
 35261453,
 35421060,
 34982242,
 35149612,
 35297881,

In [70]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {
  'domain': 'zigbang',
  'withCoalition': 'true',
  'item_ids': ids[:900], # 아이템 데이터의 갯수를 999개까지 사용 가능
}
response = requests.post(url, params)
response

<Response [200]>

In [71]:
pd.options.display.max_columns = 50 # ... 없애서 전부 보기

In [72]:
# json을 list > dict로 바꿔주고 df로
data = response.json()['items']
df = pd.DataFrame(data)
df.tail(2) # tail은 index값과 전체 데이터 갯수를 알 수 있기 때문에 많이 보심
# 여기까지가 크롤링은 끝난 걷네
# 필요한 데이터 필터링이 필요함

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,contract
673,None,35469483,https://ic.zigbang.com/ic/items/35469483/1.jpg,월세,월세,500,45,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,1,1,4,♥저렴한 풀옵션 원룸♥전용넓어요!♥주차가능♥오늘 나온 매물,None,01,마포구 중동,"{'lat': 37.57037771910298, 'lng': 126.90410117...",False,True,원룸,[],서울시 마포구 중동,None,None,5,2023-02-17T10:29:03+09:00,True,
674,None,35442758,https://ic.zigbang.com/ic/items/35442758/1.jpg,전세,전세,21000,0,59.93,"{'m2': 59.93, 'p': '18.1'}","{'m2': 51.29, 'p': '15.5'}",None,None,2,2,2,"⭐대출,보증보험OK⭐올리모델링⭐역세권⭐투룸⭐",None,04,마포구 중동,"{'lat': 37.57071480001094, 'lng': 126.90475250...",False,True,빌라,[],서울시 마포구 중동,None,None,1,2023-02-16T11:23:15+09:00,True,


In [73]:
# 필요한 데이터 뽑기 전 일일히 복붙 귀찮으니까 이거 쳐서 나온값 복붙해서 하나씩 지워나가자
df.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new', 'contract'],
      dtype='object')

In [74]:
colums = ['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 'floor', 'building_floor', 'title', 'address', 'status', 'service_type', 'tags', 'address1', 'manage_cost']
df = df[colums] 
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,title,address,status,service_type,tags,address1,manage_cost
673,35469483,월세,500,45,19.83,1,4,♥저렴한 풀옵션 원룸♥전용넓어요!♥주차가능♥오늘 나온 매물,마포구 중동,True,원룸,[],서울시 마포구 중동,5
674,35442758,전세,21000,0,59.93,2,2,"⭐대출,보증보험OK⭐올리모델링⭐역세권⭐투룸⭐",마포구 중동,True,빌라,[],서울시 마포구 중동,1


# 데이터 전처리)

In [75]:
address

'망원동'

In [76]:
# 리셋인덱스 드롭 트루는 정렬되는 것 
colums = ['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 'floor', 'building_floor', 'title', 'address', 'status', 'service_type', 'tags', 'address1', 'manage_cost'] 
df = df[colums] 
df = df[df['address'].str.contains(address)].reset_index(drop=True) 
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,title,address,status,service_type,tags,address1,manage_cost
87,35197484,월세,200,60,16.53,3,3,💖즉시입주💖나오면바로계약💖실매물💖선착순매물💖,마포구 망원동,True,원룸,[],서울시 마포구 망원동,6
88,35474326,월세,1000,80,46.28,2,3,🔮마포구청역30초🧧,마포구 망원동,True,빌라,[추천],서울시 마포구 망원동,2


## 꿀팁)

- 서점에 나와 있는 책들은 99% 인문책이다.
- 그 책을 개념을 배우긴 쉽지만, 현업을 하긴 힘들다. 
- 최신 서적을 봐야 함. 1년만 지나도 코드 작성한 게 진행 안 될 수도 있어서.
- 한 달 안에 본다.
- pandas의 공식 사이트를 본다. 책에 없는 내용들이 있는데 양이 엄청 방대하다
- 다 해볼 필요 없이 아 이런 거 있구나~ 하니까 한번 쭈욱 읽어봄
- 10 minutes to pandas 읽어보고 실습해보기 
https://pandas.pydata.org/docs/user_guide/10min.html

### 파이썬에서?
- pep documents >> 100개가 넘음. python을 잘 사용하게 해주는 문서
- pep20, pep8 > 파이썬의 철학, 파이썬 코딩 스타일 가이드
- 회사 가면 pep8에 어긋났을 때 githup, gitlab에 업로드 x
- vscode나 뭐 등등에 pep8 교정해주는 기능도 있음4
- pep8 : 문법, 컨벤션(코드 가독성 및 허용 범위..?) 체크
- flake8 : 코드효율성 체크 (if, for문 남발하면 업로드 x)

In [77]:
# 파이썬의 철학 >> 파이썬스러운 코드
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


- 코드를 작성할 땐 문법(에러발생 o, 코드실행 x), 컨벤션(에러발생 x, 코드실행 o)
  - 1data = 1 >> 식별자 앞에서는 숫자를 쓸 수 없다. 문법이 틀린 것
  - data=1 >> 컨벤션에는 어긋남. 하지만 실행은 됨 >>> 연산자 앞 뒤로 공백을 한칸씩 띄우라고 파이썬 철학에 의거해서 만들어진 거임

# 스페셜 커맨드(매직커멘드) : %, %% : ipython에서 제공하는 커멘드
- %ls 현재 디렉토리 안에 있는 파일
- %pwd 현재 작성하고 있는 파일 경로
- %%time 셀이 수행되는 시간

# 쉘 커멘드 : ! : os system에서 제공하는 커멘드
- !pip install geohash2 해도 cli환경에서 입력한 것처럼 할 수 있음